In [1]:
import os
from fairseq.examples.speech_to_text.data_utils import load_df_from_tsv, save_df_to_tsv

[2024-02-06 05:13:48,088] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/siqiouya/anaconda3/envs/sllama/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dirname = '/data/user_data/siqiouya/dataset/must-c-v1.0/en-es'
split = 'train_mfa'
df = load_df_from_tsv(os.path.join(dirname, split + '.tsv'))
# new_paths = []
# for a in df['audio']:
#     new_paths.append('/scratch/siqiouya/dataset/must-c-v1.0/' + a)
# df['audio'] = new_paths
# save_df_to_tsv(df, os.path.join(dirname, split + '.tsv'))

In [10]:
df[df['text_word'] == '']

,id,audio,n_frames,speaker,src_text,tgt_text,src_lang,tgt_lang,speech_word,text_word
11,ted_1_11,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,137600,spk.1,And I told it pretty much the same way I've ju...,Y la conté bastante del mismo modo en que la a...,en,es,,
19,ted_1_19,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,228960,spk.1,"(Laughter) And the story began, ""Former Vice P...","Y la historia comenzaba, ""El ex vicepresidente...",en,es,,
20,ted_1_20,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,279200,spk.1,(Laughter) Before I could get back to U.S. soi...,Antes de que pudiera regresar a suelo norteame...,en,es,,
22,ted_1_23,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,3040,spk.1,Chris Anderson: It's a deal! (Applause),Chris Anderson: ¡Es un trato!,en,es,,
24,ted_1_25,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,175840,spk.1,I want to start with a couple of — I'm going t...,Quiero empezar con — voy a mostrar unas nuevas...,en,es,,
...,...,...,...,...,...,...,...,...,...,...
259948,ted_12908_51,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,1120,spk.12908,(Laughter),(Risas),en,es,,
259949,ted_12908_52,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,8160,spk.12908,That said —,El tuit decía:,en,es,,
259967,ted_12908_72,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,85440,spk.12908,"And you should know that this idea, this hope ...","Y debo decir que esta idea, esta esperanza, no...",en,es,,
260028,ted_13591_32,/scratch/siqiouya/dataset/must-c-v1.0/en-es/da...,206880,spk.13591,"If women have led countries, women have gone t...","Si hay mujeres que han liderado países, que ha...",en,es,,


# Debug Train

In [ ]:
update_config = os.path.join(, 'config_large.json')

# Change path of tsv

In [4]:
import os
from fairseq.examples.speech_to_text.data_utils import load_df_from_tsv, save_df_to_tsv

In [2]:
dev = 'train'

In [5]:
df = load_df_from_tsv(os.path.join('/scratch/siqiouya/dataset/must-c-v1.0/en-es/', '{}.tsv'.format(dev)))

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/siqiouya/dataset/must-c-v1.0/en-es/train.tsv'

# Mask

In [1]:
import torch

In [2]:
def generate_2d_causal_mask(seq_len, dtype, device='gpu', blocksize=1):
    """
    Generates a 2D causal mask for multi-head attention.
    
    Args:
        seq_len (int): The length of the sequence.
        dtype (torch.dtype): The data type for the mask.
        device (str): The device on which to create the mask.
    
    Returns:
        torch.Tensor: A 2D causal attention mask.
    """
    blocksizes = [min(blocksize, seq_len - i * blocksize) for i in range((seq_len + blocksize - 1) // blocksize)]
    blocks = [torch.ones((s, s), device=device, dtype=dtype) for s in blocksizes]
    mask = torch.block_diag(*blocks)

    tril_row, tril_col = torch.tril_indices(seq_len, seq_len)
    mask[tril_row, tril_col] = 1

    mask.masked_fill_(mask == 0, float('-inf'))
    mask.masked_fill_(mask == 1, 0)

    return mask

In [3]:
generate_2d_causal_mask(6, float, device='cpu', blocksize=4)

tensor([[0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]], dtype=torch.float64)